# Suchoptionen Metadaten

In [1]:
import pandas as pd

"""Einlesen der Metadaten-Tabelle als Dataframe"""
df_metadata = pd.read_csv("files\metadata_letters.csv")

def list_content(column_name):
    """Auflisten der unique-Werte einer Spalte des Dataframe in alphabetischer Reihenfolge"""
    names = (df_metadata[column_name].unique())
    for i in sorted(names):
        print(i)

def print_result(column_name, name):
    """Ausgabe der Zeilen des Dataframe, die ein Suchwort (= name) enthalten"""
    result = df_metadata.loc[df_metadata[column_name] == name, "Dateiname"].to_list()
    for i in result:
        print(i)

## Suche nach Absender*innen

### Ausgeben einer Liste von Absender*innen

In [2]:
list_content("Absender*in")

Aapeli Saarisalo
Benno Landsberger
Cyril John Gadd
François Thureau-Dangin
Friedrich Stummer
Giorgio Castellino
Harri Holma
Jakob J. Finkelstein
Joh. F. Kunstmann
Johannes Pedersen
Karl Budde
Karl Friedrich Müller
Reginald Campbell Thompson
Richard Hartmann
Walter Andrae
Walter G. Kunstmann


### Auswahl eines Namens und Ausgabe der zugehörigen Briefe

In [3]:
#Einfügen des gesuchten Namens
name = "Walter Andrae"

print_result("Absender*in", name)

1932-06-30_Andrae_Landsberger


## Suche nach Empfänger*innen

### Ausgeben einer Liste von Empfänger*innen

In [4]:
list_content("Empfänger*in")

Benno Landsberger
François Thureau-Dangin
Jussi Aro
Karl Friedrich Müller
Torgny Segerstedt


### Auswahl eines Namens und Ausgabe der zugehörigen Briefe

In [5]:
#Einfügen des gesuchten Namens
name = "Jussi Aro"

print_result("Empfänger*in", name)

1960-09-27_Landsberger_Aro


## Suche nach Aufenthaltsorten der Absender*innen

### Ausgeben einer Liste von Aufenthaltsorten

In [6]:
list_content("Aufenthaltsort Absender*in")

Ankara
Berkeley
Berlin
Chicago
Frydek
Heidelberg
Helsinki 
Kopenhagen
Leipzig
London
Marburg
Oxford
Paris
Porto Alegre
Rom
Wiederitzsch 
Würzburg


### Auswahl eines Ortes und Ausgabe der zugehörigen Briefe

In [7]:
#Einfügen des gesuchten Ortes
name = "Leipzig"

print_result("Aufenthaltsort Absender*in", name)

1929-07-10_Mueller_Landsberger


## Suche nach einem Datumszeitraum

In [8]:
#Einfügen eines Datumszeitraums
start_date = "1929-01-01"
end_date = "1931-01-01"

"""Umwandeln des Datums in ein Datetime-Format; Definieren des Datumszeitraums"""
df_metadata["Datum (ISO 8601)"] = pd.to_datetime(df_metadata["Datum (ISO 8601)"], format="ISO8601")
mask = (df_metadata["Datum (ISO 8601)"] > start_date) & (df_metadata["Datum (ISO 8601)"] <= end_date)

result_date = df_metadata.loc[mask, "Dateiname"].to_list()
for i in result_date:
    print(i)

1929-06-06_Landsberger_ThureauDangin
1929-07-10_Mueller_Landsberger
1929-12-09_Hartmann_Landsberger
1930-01-03_Budde_Landsberger


## Suche nach der Briefsprache

### Ausgeben einer Liste von Sprachen

In [9]:
list_content("Sprache")

deutsch
englisch
französisch


### Auswahl einer Sprache und der zugehörigen Briefe

In [10]:
#Einfügen der gesuchten Sprache
name = "englisch"

print_result("Sprache", name)

1933-03-08_Gadd_Landsberger
1943-10-01_Thompson_Landsberger
1956-02-15_Landsberger_Segerstedt
1960-09-27_Landsberger_Aro
1962-02-19_Finkelstein_Landsberger


# Suchoptionen Briefinhalte

In [11]:
import glob
import pandas as pd
import re
from bs4 import BeautifulSoup
from os.path import join

"""Die CSV-Datei references_names.csv enthält eine Liste der ref-Attribute und xml:lang-Attribute
(= Links zu Wikidata, Pleiades, GND, Sprachenkennung) (bzw. des Tag-Inhalts, falls kein ref-Attribut existiert),
die in den Kerntexten der Briefe den jeweiligen Tags zugeteilt wurden.
Die CSV-Datei wird als Dataframe eingelesen und die beiden Spalten in einem Dictionary 
als key-value-Paare zusammengefasst"""
df_ref_names = pd.read_csv("files/references_names.csv")
dict_ref_names = dict(zip(df_ref_names["Ref"],df_ref_names["Name"]))

def content_search(tag, attribute, ref):
    """Einlesen der in TEI kodierten Textinhalte;
    Auffinden der ausgesuchten Tags und Attribute;
    Entfernen von Zeilenumbrüchen und doppelten Leerzeichen bei den Tag-Inhalten;
    Gemeinsames Speichern der ref-Attribute/xml:lang-Attribute und Briefnamen, Entfernen von Duplikaten;
    Zusammenführen der Suchergebnisse für jeden einzelnen Brief in einem gemeinsamen Dictionary 
    mit ref-Attribut/xml:lang-Attribut und Briefname;
    Abgleich der ref-Attribute/xml:lang-Attrbute mit den in der CSV-Datei gesammelten Attributen und 
    Ersetzen der Attribute durch ein Label;
    Ausgabe des Ergebnis im Format 'Label : Briefname(, Briefname, etc.)'
    """
    tags_all_letters = []
    ref_filename_dict = {}
    for file in glob.glob(join("files", "letters", "tei", "*.xml")):
        with open(file, "r", encoding="utf-8") as infile:
            text = infile.read()
            filename = file.split("\\")[-1].split(".")[0]
            soup = BeautifulSoup(text, "xml")
            tag_list = soup.find_all(tag, attribute)
            ref_filename_list = []  
            for i in tag_list:
                content = i.text
                content = re.sub("-\n", "", content) #Zusammenführung von Wörtern, die am Zeilenende durch Bindestrich getrennt werden
                content = re.sub("\n", " ", content) #Zusammenführung von Bezeichnungen, die aus mehreren Wörtern bestehen und auf zwei oder mehr Zeilen aufgeteilt sind
                content = re.sub("  ", "", content)  #Entfernen von doppelten Leerzeichen
                if i.has_attr(ref):
                    ref_filename_list.append((i[ref], filename))
                else:
                    ref_filename_list.append((content, filename))
            ref_filename_list = list(set(ref_filename_list))
            for i in ref_filename_list:
                tags_all_letters.append(i)
    for (y,z) in tags_all_letters:
        ref_key = y
        ref_value = z
        if y in ref_filename_dict:
            ref_filename_dict[y] += "; " + z
        else:
            ref_filename_dict[y] = z

    label_filename_dict = dict((dict_ref_names[key], value) for (key,value) in ref_filename_dict.items())
    label_filename_dict = dict(sorted(label_filename_dict.items()))
    for k,v in label_filename_dict.items():
        print(f"{k} : {v}")

### Erwähnungen von modernen Personen

In [12]:
content_search("persName", {"type" : "modern"}, "ref")

Albert Schott : 1929-12-09_Hartmann_Landsberger
Albrecht Goetze : 1929-12-09_Hartmann_Landsberger
Alfred Pohl : 1937-01-29_Mueller_Landsberger; 1937-02-09_Landsberger_Mueller
Anton Moortgat : 1937-02-09_Landsberger_Mueller
Cyril John Gadd : 1931-03-11_ThureauDangin_Landsberger; 1936-05-05_Landsberger_Mueller
Felix Ernst Peiser : 1937-01-29_Mueller_Landsberger
Hans Ehelolf : 1932-06-30_Andrae_Landsberger
Hans-Siegfried Schuster : 1936-05-05_Landsberger_Mueller; 1937-01-29_Mueller_Landsberger; 1937-02-09_Landsberger_Mueller
Heinrich Zimmern : 1931-03-11_ThureauDangin_Landsberger; 1932-03-22_Pedersen_Landsberger; 1937-01-29_Mueller_Landsberger; 1937-04-12_Castellino_Landsberger
Henri Frankfort : 1937-02-09_Landsberger_Mueller
Johannes Friedrich : 1937-01-29_Mueller_Landsberger
Johannes Hehn : 1933-04-17_Stummer_Landsberger
Paul Koschaker : 1937-02-09_Landsberger_Mueller
Stephen Herbert Langdon : 1931-03-11_ThureauDangin_Landsberger; 1937-02-09_Landsberger_Mueller
Sébastien Ronzevalle : 19

### Erwähnungen von Konferenzen/Kongressen

In [13]:
content_search("title", {"type" : "meeting"}, "ref")

International Congress of Orientalists : 1937-04-12_Castellino_Landsberger


### Erwähnungen von modernen Orten

In [14]:
content_search("placeName", {"type" : "modern"}, "ref")

Deutschland : 1932-03-22_Pedersen_Landsberger; 1936-05-05_Landsberger_Mueller
England : 1936-05-05_Landsberger_Mueller
Hamah : 1932-03-22_Pedersen_Landsberger
Israel : 1932-03-22_Pedersen_Landsberger
Leiden : 1932-03-22_Pedersen_Landsberger
Leipzig : 1929-12-09_Hartmann_Landsberger; 1931-03-11_ThureauDangin_Landsberger; 1932-03-22_Pedersen_Landsberger
London : 1932-03-22_Pedersen_Landsberger; 1936-05-05_Landsberger_Mueller; 1937-02-09_Landsberger_Mueller
Marburg : 1929-12-09_Hartmann_Landsberger
Polen : 1936-05-05_Landsberger_Mueller
Rom : 1937-04-12_Castellino_Landsberger
Tschechoslowakei : 1936-05-05_Landsberger_Mueller
Österreich : 1936-05-05_Landsberger_Mueller


### Erwähnungen von modernen Institutionen

In [15]:
content_search("orgName", {"type" : "academy"}, "ref")

Sächsische Akademie der Wissenschaften : 1932-06-30_Andrae_Landsberger


In [16]:
content_search("orgName", {"type" : "institute"}, "ref")

Orientalisches und Sprachwissenschaftliches (indogermanistisches) Seminar Universität Heidelberg : 1929-12-09_Hartmann_Landsberger


### Erwähnungen von altorientalischen Orten

In [17]:
content_search("placeName", {"type" : "ANE"}, "ref")

Assur : 1929-07-10_Mueller_Landsberger
Assyrisches Reich : 1929-07-10_Mueller_Landsberger
Babylon : 1937-01-29_Mueller_Landsberger
Ninive : 1937-01-29_Mueller_Landsberger
Ugarit : 1931-03-11_ThureauDangin_Landsberger


### Erwähnungen von altorientalischen Göttern

In [18]:
content_search("name", {"type" : "deity"}, "ref")

Assur : 1929-07-10_Mueller_Landsberger
Enlil : 1929-07-10_Mueller_Landsberger
Marduk : 1929-07-10_Mueller_Landsberger
Mārat-nāri : 1937-01-29_Mueller_Landsberger
Nanaya : 1937-04-12_Castellino_Landsberger
Ninlil : 1929-07-10_Mueller_Landsberger
Serua : 1929-07-10_Mueller_Landsberger


### Erwähnungen von Forschungspublikationen

In [19]:
content_search("title", {"type" : "journal"}, "ref")

MDOG : 1929-07-10_Mueller_Landsberger
MFOB : 1932-03-22_Pedersen_Landsberger
RA : 1931-03-11_ThureauDangin_Landsberger
ZA : 1933-04-17_Stummer_Landsberger; 1937-02-09_Landsberger_Mueller


In [20]:
content_search("title", {"type" : "monograph"}, "ref")

Falkenstein/Haupttypen der sumerischen Beschwörung : 1931-03-11_ThureauDangin_Landsberger
Landsberger/Die serie ana ittišu : 1931-03-11_ThureauDangin_Landsberger; 1937-01-29_Mueller_Landsberger
Landsberger/Fauna des alten Mesopotamien : 1932-06-30_Andrae_Landsberger
Landsberger/Serie Ur-e-a= nâqu : 1937-02-09_Landsberger_Mueller
Thureau-Dangin/Rituels accadiens : 1937-01-29_Mueller_Landsberger
Zimmern/BBR : 1937-01-29_Mueller_Landsberger
Zimmern/Babylonisches Neujahrsfest 2 : 1937-02-09_Landsberger_Mueller


### Erwähnungen von Keilschrifttexten

In [21]:
content_search("title", {"type" : "cuneiformtext"}, "ref")

K 10209 : 1937-01-29_Mueller_Landsberger; 1937-02-09_Landsberger_Mueller
K 3438a : 1937-01-29_Mueller_Landsberger; 1937-02-09_Landsberger_Mueller
K 3455 : 1937-01-29_Mueller_Landsberger
KAR 135+ : 1929-07-10_Mueller_Landsberger; 1937-01-29_Mueller_Landsberger
KAR 139 : 1929-07-10_Mueller_Landsberger
KAR 146 : 1937-01-29_Mueller_Landsberger
KAR 215+ : 1937-01-29_Mueller_Landsberger
Lu A : 1937-02-09_Landsberger_Mueller
Rm 2, 36 : 1931-03-11_ThureauDangin_Landsberger
Sargons 8. Feldzug : 1929-07-10_Mueller_Landsberger
Ura=hubullu Tafel 1 : 1936-05-05_Landsberger_Mueller
VS 10, 215 : 1937-04-12_Castellino_Landsberger


### Erwähnungen von altorientalischen Textpassagen oder Lemmata

In [22]:
import glob
import re
from bs4 import BeautifulSoup
from os.path import join

def search_foreigntext(tag, attribute):
    """Einlesen der in TEI kodierten Textinhalte;
    Auffinden von ausgesuchten Tags und Attributen;
    Ausgabe des Ergebnis im Format 'Gesuchtes Tag: Briefname'
    """
    for file in glob.glob(join("files", "letters", "tei", "*.xml")):
        with open(file, "r", encoding="utf-8") as infile:
            text = infile.read()
            filename = file.split("\\")[-1].split(".")[0]
            soup = BeautifulSoup(text, "xml")
            tag_list = soup.find_all(tag, attribute)
            tag_tuple_list = []
            for i in tag_list:
                i = i.text
                i = re.sub("-\n", "", i) #Zusammenführung von Wörtern, die am Zeilenende durch Bindestrich getrennt werden
                i = re.sub("\n", " ", i) #Zusammenführung von Bezeichnungen, die aus mehreren Wörtern bestehen und auf zwei oder mehr Zeilen aufgeteilt sind
                i = re.sub("  ", "", i)  #Entfernen von doppelten Leerzeichen
                i = (i, filename)
                tag_tuple_list.append(i)
            tag_tuple_list = list(set(tag_tuple_list))
            tag_tuple_list.sort(key=lambda t : tuple(t[0].lower()))
            for x,y in tag_tuple_list:
                print(x + ": " + y)

### Akkadisch

In [23]:
search_foreigntext("foreign", {"xml:lang" : "akk"})

ab-ba-sa: 1929-07-10_Mueller_Landsberger
abnu: 1929-07-10_Mueller_Landsberger
ip-pal: 1929-07-10_Mueller_Landsberger
kararu: 1929-07-10_Mueller_Landsberger
Pû-lišānu: 1929-07-10_Mueller_Landsberger
lapātu: 1936-05-05_Landsberger_Mueller
lapātu ša nakāsi: 1936-05-05_Landsberger_Mueller
naru: 1937-01-29_Mueller_Landsberger
takultu: 1937-01-29_Mueller_Landsberger
il-ta-am: 1937-04-12_Castellino_Landsberger


### Erwähnungen von altorientalischen Sprachen

In [24]:
content_search("lang", "", "xml:lang")

Akkadisch : 1929-12-09_Hartmann_Landsberger; 1931-03-11_ThureauDangin_Landsberger; 1932-03-22_Pedersen_Landsberger; 1936-05-05_Landsberger_Mueller; 1937-01-29_Mueller_Landsberger; 1937-02-09_Landsberger_Mueller; 1937-04-12_Castellino_Landsberger
Aramäisch : 1929-07-10_Mueller_Landsberger; 1932-03-22_Pedersen_Landsberger
Hebräisch : 1937-04-12_Castellino_Landsberger
Sumerisch : 1931-03-11_ThureauDangin_Landsberger; 1932-03-22_Pedersen_Landsberger
